In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

# Load preprocessed data from previous notebook using pickle
with open('step3.pkl', 'rb') as f:
    data_bundle = pickle.load(f)

X = data_bundle['X']
preprocessor = data_bundle['preprocessor']
customer_features = data_bundle['customer_features']
df = data_bundle['df']

print("Preprocessed data loaded from step3.pkl:", X.shape)
print("Data with features loaded:", df.shape)

# Clustering (ML Part)
Apply K-Means clustering, try different k values (Elbow Method), and validate clusters with Silhouette Score.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
# Elbow Method
inertia = []
K = range(2, 10)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# Fit KMeans and assign clusters
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(X)
df['cluster'] = clusters

In [ ]:
# Silhouette Score
score = silhouette_score(X, clusters)
print('Silhouette Score:', score)

In [ ]:
# Save clustered data for next notebook using pickle
final_data_bundle = {
    'df': df,
    'customer_features': customer_features,
    'X': X,
    'kmeans_model': kmeans
}

with open('step4.pkl', 'wb') as f:
    pickle.dump(final_data_bundle, f)

print("Data with clusters saved as step4.pkl")